## Shearching for a Shkulk Shrkieaker

I wanna know if there's a live Sculk Shrieker under my base (and if not I want to reset the chunklets under my base until I get one)

## Imports, Setup and Macros

In [1]:
import json
from collections import Counter
from os import environ
from pathlib import Path
from typing import Any, Collection, Dict, List, Tuple

import mutf8
import pandas as pd
from IPython.display import Markdown, display
from nbt import nbt, region

In [2]:
def format_file_size(path: Path) -> str:
    """Print the size of the specified file in
    human-readible form (KB / MB / GB)

    Parameters
    ----------
    path : Path
        The path to the file

    Returns
    -------
    str
        A prettily formatted file size

    Notes
    -----
    I would be shocked if there isn't a utility already built
    into the standard library to do this, but all I could find
    via Googling was a bunch of recipes and examples
    """
    size: float = path.stat().st_size  # in bytes
    for unit in ("B", "KB", "MB", "GB"):
        if size < 1024 / 2:
            return f"{size:.1f} {unit}"
        size = size / 1024
    return f"{size} TB"

In [3]:
def summarize_keystore(keystore: Dict[str, Any]) -> None:
    """Display a summary of the contents of a key-value store

    Parameters
    ----------
    keystore : dict
        The keystore to summarize

    Returns
    -------
    None
    """

    def _summarize_keystore(keystore: Dict[str, Any]) -> str:
        summary = ""
        for k, v in keystore.items():
            summary += f"\n - `{k}` : "
            if isinstance(v, (str, nbt.TAG_String)):
                summary += f'`"{v}"`'
            elif not isinstance(v, Collection):
                summary += f"`{str(v)}`"
            else:
                length = len(v)
                if 0 < length < 3:
                    summary += "\n"
                    if not isinstance(v, Dict):
                        v = {i: item for i, item in enumerate(v)}
                    summary += "\n".join(
                        (f"\t{line}" for line in _summarize_keystore(v).split("\n"))
                    )
                else:
                    summary += f"({len(v)} items)"
        return summary

    display(Markdown(_summarize_keystore(keystore)))

In [4]:
save_folder = Path(environ["SAVE_PATH"])

# make sure this is set correctly
for path in sorted(save_folder.glob("*")):
    print(f"- {path.name} ({'folder' if path.is_dir() else format_file_size(path)})")

- DIM-1 (folder)
- DIM1 (folder)
- advancements (folder)
- data (folder)
- datapacks (folder)
- entities (folder)
- icon.png (8.6 KB)
- level.dat (21.0 KB)
- level.dat_old (21.0 KB)
- level11326231134829879582.dat (0.0 B)
- level14927678268100607923.dat (0.0 B)
- level1786655981796876926.dat (0.0 B)
- level4463453738642305340.dat (0.0 B)
- level6425531070021529407.dat (0.0 B)
- level7443636089696258371.dat (0.0 B)
- level8832581565660323154.dat (0.0 B)
- playerdata (folder)
- poi (folder)
- region (folder)
- serverconfig (folder)
- session.lock (3.0 B)
- stats (folder)
- worldgen_settings_export.json (1.5 KB)


## Search the Region Files

I'm looking in the area bounded by (-1600, -4000) and (-1400, -3750)

That's `r.-4.-8.mca` and `r.-3.-8.mca`

In [5]:
region_files = (
    save_folder / "region" / "r.-4.-8.mca",
    save_folder / "region" / "r.-3.-8.mca",
)

In [the Pigstep search](Pigstep.ipynb) we dug into how block entities are handled in the chunk data, so let's start by seeing if we can find some Shkriekers listed as block entities.

In [6]:
shkriekers = []
for region_file in region_files:
    region_data = region.RegionFile(region_file)
    for chunk in region_data.iter_chunks():
        for entity in chunk["block_entities"]:
            if "shrieker" in entity["id"].value.lower():
                shkriekers.append(entity)
                print("Found a shkrieker")

Found a shkrieker
Found a shkrieker
Found a shkrieker
Found a shkrieker


Oh cool! Especially since I don't think I placed any harvested ones anywhere around my base.

In [7]:
summarize_keystore(shkriekers[0])


 - `listener` : (4 items)
 - `z` : `-3855`
 - `x` : `-1545`
 - `id` : `"minecraft:sculk_shrieker"`
 - `y` : `-51`
 - `keepPacked` : `0`
 - `warning_level` : `0`

Hmm... well _I_ know this is live ('cuz I definitely didn't place anything at y=-51) but let's see if I can find the key in the data that reflects that. 

In [8]:
summarize_keystore(shkriekers[0]["listener"])


 - `source` : 
	
	 - `0` : `"type"`
	 - `1` : `"pos"`
 - `range` : `8`
 - `event_distance` : `0.0`
 - `event_delay` : `0`

In [9]:
summarize_keystore(shkriekers[0]["listener"]["source"])


 - `type` : `"minecraft:block"`
 - `pos` : (3 items)

Aw, to heck with it. Let's just get the coords of the four shkriekers, and I can get to work on building a house (and bubble column) for a warden.

In [10]:
for shkrieker in shkriekers:
    print("Shkrieker at" f" ({shkrieker['x']}, {shkrieker['y']}, {shkrieker['z']})")

Shkrieker at (-1545, -51, -3855)
Shkrieker at (-1463, -41, -3970)
Shkrieker at (-1469, -44, -3909)
Shkrieker at (-1455, -54, -3885)
